In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
print ("DISCLAIMER")
print ("for this idea to work, the wallet template code file must be edited to change the output during the print to string representation of the wallet")
print ("C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Python39_64\\Lib\\site-packages\\xrpl\\wallet\\main.py")
print (" ")
print ('f"public_key: {self.public_key}",\nf"private_key in hex format: {self.private_key}",\nf"seed: {self.seed}",\nf"classic_address: {self.classic_address}",')
print ("\n \n ")

# Define the network client
from xrpl.clients import JsonRpcClient
JSON_RPC_URL = "https://s.altnet.rippletest.net:51234/"
client = JsonRpcClient(JSON_RPC_URL)
import xrpl


##test server functionality to fund a wallet for testing purposes
#from xrpl.wallet import generate_faucet_wallet
#autoGeneratedWallet = generate_faucet_wallet(client, debug=True)
#print (autoGeneratedWallet)


#public_key: EDBA1368AF1355CBB24EEB4C390DE76FF670178359BC55D8352BF820A9B4669D7F
#private_key in hex format: EDDA1538FBF543D7097F1B472638BFC9B1EA8BB92EC22C473FD2C1E026BA0853
#seed: sEdSABA4b6W5oJLeoLiz6zevtyvny1m
#classic_address: rEpphhSrTzQr1fTfJ73UE6tHZ6ejC3DT3o




##if you know the seed, you can create a wallet with this code, find the sequence with account_info() but it isn't used anyway
#from xrpl.wallet import Wallet
#test_wallet = Wallet(seed="shq3PGyrFjWntfSXxt56BmwS1aAVa", sequence=28718475)
#print(test_wallet.classic_address) 
#print (" ")
#print(test_wallet) 
#print (" ")


import datetime
now = datetime.datetime.now()
#print("Current date and time: ")
#print(str(now))
from xrpl.wallet import Wallet
from xrpl.core.keypairs import derive_classic_address



inputStr = input("\nto create a new wallet, type createWallet \nto transfer XRP to a wallet type transfer:\n")

if inputStr == "createWallet":

    #this newWalletToBeUsed will be a permanent wallet after being initialized
    newWalletToBeUsed = Wallet.create()
    print ("\n\nnew wallet info:")
    print ("this information is also printed to a file in the directory of the python project")
    print (newWalletToBeUsed)
    print (str(now), "\n", newWalletToBeUsed, "\n--------------------", file=open('XRP_wallet_value.txt','a'))
    quit()
    
    
elif inputStr == "transfer":
    publicKeyStr = input("Enter public key of the account SENDING the XRP: ")
    privateKeyStr = input("Enter private key of the account SENDING the XRP: ")
    destination_address = input("enter the XRP destination address (classic format): ")
    destinationTagStr = input("Enter Destination Tag number (0 if NONE): ")
    amountToSendStr = input("Enter the amount of XRP to send: ")

    print (" ")

    #we create a temporary wallet objet that will be used for our XRP account that we have the pub/private keys from mnemonic bip39
    test_wallet = Wallet.create()
    print ("before the hard coding:")
    print (test_wallet)
    
    
    print (" ")


    #we hard code the public and private keys from the mnemonic bip39 info
    test_wallet.public_key = publicKeyStr
    test_wallet.private_key = privateKeyStr
    test_wallet.classic_address = derive_classic_address(test_wallet.public_key)
    print ("after the hard coding:")
    print ("seed value hasnt changed(but it also isn't used in the transaction object)")
    print (test_wallet)
    
    
    print (" ")


    # Prepare transaction ----------------------------------------------------------
    my_payment = xrpl.models.transactions.Payment(
        account=test_wallet.classic_address,
        amount=xrpl.utils.xrp_to_drops(int(amountToSendStr)),
        destination=destination_address,
        #destination tag field is optional
        destination_tag=int(destinationTagStr)
        )
    print ("Payment object:", my_payment)
    
    
    print (" ")
    

    # Sign transaction -------------------------------------------------------------
    signed_tx = xrpl.transaction.safe_sign_and_autofill_transaction(
            my_payment, test_wallet, client)
    max_ledger = signed_tx.last_ledger_sequence
    tx_id = signed_tx.get_hash()
    print ("Signed transaction:", signed_tx)
    print ("Transaction cost:", xrpl.utils.drops_to_xrp(signed_tx.fee), "XRP")
    print ("Transaction expires after ledger:", max_ledger)
    print ("Identifying hash:", tx_id)



    # Submit transaction -----------------------------------------------------------
    try:
        tx_response = xrpl.transaction.send_reliable_submission(signed_tx, client)
    except xrpl.transaction.XRPLReliableSubmissionException as e:
        exit(f"Submit failed: {e}")

    # Wait for validation ----------------------------------------------------------
    # send_reliable_submission() handles this automatically, but it can take 4-7s.

    # Check transaction results ----------------------------------------------------
    import json
    print (json.dumps(tx_response.result, indent=4, sort_keys=True))
    print (f"Explorer link: https://testnet.xrpl.org/transactions/{tx_id}")
    metadata = tx_response.result.get("meta", {})
    if metadata.get("TransactionResult"):
        print ("Result code:", metadata["TransactionResult"])
    if metadata.get("delivered_amount"):
        print ("XRP delivered:", xrpl.utils.drops_to_xrp(
                    metadata["delivered_amount"]))
    
else:
    print ("wrong input")
    quit()


   


DISCLAIMER
for this idea to work, the wallet template code file must be edited to change the output during the print to string representation of the wallet
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\Lib\site-packages\xrpl\wallet\main.py
 
f"public_key: {self.public_key}",
f"private_key in hex format: {self.private_key}",
f"seed: {self.seed}",
f"classic_address: {self.classic_address}",

 
 



to create a new wallet, type createWallet 
to transfer XRP to a wallet type transfer:
 createWallet




new wallet info:
this information is also printed to a file in the directory of the python project
public_key: ED0177CA23206712ECDEFC99A99C4BA6473B98A5B0639881D29B5418459F745253
private_key: -HIDDEN-
classic_address: r9dMGsPsckRnhBe6aQMaZZeVunjUntzNNm
